In [1]:
from typing import Callable, Tuple

import numpy as np
import pandas as pd
from numba import prange, njit, typed, types
from tqdm.notebook import trange
from numpy.typing import NDArray
from numba import njit, prange
import matplotlib.pyplot as plt
from scipy import stats

from util import normalize_values, ideal_point_method, linear_convolution

In [2]:
def get_service_times() -> NDArray['float']:
    df = pd.read_csv('../data/service_times.csv')
    df.rename(columns={'TTI 0.95': 'service_time' }, inplace=True)
    df['service_time'] = df['service_time'].map(lambda time: float(time.replace('ms', '')))

    return np.array(df['service_time'])

In [3]:
service_times = get_service_times()

In [4]:
@njit
def arrive_cdf(n: int) -> float:
    return np.random.gamma(n, 0.299)
    
@njit
def leave_cdf() -> float:
    return np.random.exponential(510)

@njit
def departure_cdf() -> float:
    mean = np.mean(service_times)
    return np.random.exponential(mean * 0.95 / 4)
    # return np.random.choice(service_times) * 0.95 / 4

In [5]:
@njit
def get_next_event(
    arrival_time: float,
    request_times_queue: list,
    departure_time: float,
) -> (float, str, int):
    event_type = 'ARRIVAL'
    min_index = -1
    min_time = arrival_time

    if departure_time != -1 and departure_time < min_time:
        event_type = 'DEPARTURE'
        min_time = departure_time

    min_leave_time_index = -1
    min_leave_time = np.inf
    if len(request_times_queue) != 0:
        min_leave_time_index = np.argmin(request_times_queue)
    if min_leave_time_index != -1:
        min_leave_time = request_times_queue[min_leave_time_index]
        
    if min_leave_time < min_time:
        min_index = min_leave_time_index
        min_time = min_leave_time
        event_type = 'LEAVE'

    return min_time, event_type, min_index

In [6]:
@njit
def arrival(
    time: float,
    num_in_system: int,
    num_total_requests: int,
    leave_indicator: int,
    request_times_queue: list,
    departure_time: float,
    n: int,
    N: int,
) -> (float, int, int, int, list, float):
    num_in_system += 1 # увеличиваем число запросов в системе
    num_total_requests += 1

    if departure_time == -1: # если сервер свободен
        mu = departure_cdf()
        departure_time = time
        departure_time = time + mu # планируем обработку запроса
    else:
        if len(request_times_queue) == N:
            num_in_system -= 1 # считаем, что заявка покинула систему
            leave_indicator += 1 # увеличиваем индикатор выхода из системы
        else:
            gamma = leave_cdf()
            request_times_queue = np.append(request_times_queue, time + gamma)
    
    alpha = arrive_cdf(n)
    time += alpha
    return time, num_in_system, num_total_requests, leave_indicator, request_times_queue, departure_time

@njit
def departure(
    time: float,
    num_in_system: int,
    request_times_queue: list,
    departure_time: float,
) -> (int, list, list):
    num_in_system -= 1 # запрос обработан - уменьшаем число запросов в системе
    
    if (len(request_times_queue) == 0): # если очередь пуста
        departure_time = -1
    else:        
        mu = departure_cdf()
        departure_time = time + mu # планируем обработку запроса
        request_times_queue = np.delete(request_times_queue, 0)
    return num_in_system, request_times_queue, departure_time

@njit
def on_leave(
    time_index: float,
    num_in_system: int,
    leave_indicator: int,
    request_times_queue: list
) -> (int, int, list):
    num_in_system -= 1 # запрос ушёл - уменьшаем число запросов в системе
    leave_indicator += 1 # увеличиваем индикатор выхода из системы
    
    request_times_queue = np.delete(request_times_queue, time_index)

    return num_in_system, leave_indicator, request_times_queue

In [7]:
@njit
def update_area_num_in_system_stat(
    time: float,
    past_event_time: float,
    num_in_system: int,
    area_num_in_system: float
) -> (float, float):
    time_since_past_event = time - past_event_time
    past_event_time = time

    area_num_in_system += (num_in_system * time_since_past_event)
    return past_event_time, area_num_in_system

In [8]:
@njit
def num_in_system_iteration(
    T: float,
    n: int,
    N: int,
    return_value: str = 'average_requests_in_system'
) -> float:
    time = 0.0
    past_event_time = 0.0

    num_in_system = 0
    area_num_in_system: float = 0.0

    num_total_requests = 0
    leave_indicator = 0
    
    arrival_time = arrive_cdf(n)
    leave = leave_cdf()

    departure_time = -1
    request_times_queue = np.empty(0, dtype=np.float64)
    
    while time < T:
        time, event_type, time_index = get_next_event(arrival_time, request_times_queue, departure_time)
        past_event_time, area_num_in_system = update_area_num_in_system_stat(
            time,
            past_event_time,
            num_in_system,
            area_num_in_system
        )
        
        if event_type == 'ARRIVAL':
            arrival_time, num_in_system, num_total_requests, leave_indicator, request_times_queue, departure_time = arrival(
                time,
                num_in_system,
                num_total_requests,
                leave_indicator,
                request_times_queue,
                departure_time,
                n,
                N
            )
        elif event_type == 'DEPARTURE':
            num_in_system, request_times_queue, departure_time = departure(
                time,
                num_in_system,
                request_times_queue,
                departure_time
            )
        else:
            num_in_system, leave_indicator, request_times_queue = on_leave(
                time_index,
                num_in_system,
                leave_indicator,
                request_times_queue
            )

    average_requests_in_system = area_num_in_system / time
    loss_probability = leave_indicator / num_total_requests

    if return_value == 'average_requests_in_system':
        return average_requests_in_system
    if return_value == 'loss_probability':
        return loss_probability

In [9]:
@njit(parallel=True)
def num_in_system_iteration_resolver(
    T: float,
    N: int,
    n: int,
    max_queue_size: int,
    x_betta: float,
    eps: float = 1e-3,
    return_value: str = 'average_requests_in_system'
) -> float:
    N_new = N
    N_old = 0

    sum_data = 0
    sum_data_sq = 0
    while True: 
        for i in prange(N - N_old):
            value = num_in_system_iteration(T, n, max_queue_size, return_value)
            sum_data += value
            sum_data_sq += value ** 2
        E = sum_data / N
        D = (sum_data_sq / N) - (E ** 2)
        N_new = int(((x_betta ** 2) * D) / (eps ** 2))

        if N_new <= N:
            return E
        N_old = N
        N = N_new
        
        print(f'New N: {N}, old N: {N_old}')

In [10]:
MAX_SERVERS = 200
MIN_SERVERS = 1

servers_num = np.arange(MIN_SERVERS, MAX_SERVERS + 1)

In [11]:
loss_probabilities = np.empty(0)

T = 10000
iterations = 1000
max_queue_size = 100
eps = 1e-3
betta = 0.95
x_betta = stats.norm.ppf(betta)

for i in trange(MIN_SERVERS, MAX_SERVERS + 1):    
    loss_probability = num_in_system_iteration_resolver(
        T,
        iterations,
        i,
        max_queue_size,
        x_betta=x_betta,
        eps=eps,
        return_value = 'loss_probability'
    )
    print(f'Probability of loss: {loss_probability} for servers number: {i}')
    loss_probabilities = np.append(loss_probabilities, loss_probability)

with open('./round-robin/loss_probabilities.npy', 'wb') as f:
    np.save(f, loss_probabilities)

  0%|          | 0/200 [00:00<?, ?it/s]

Probability of loss: 0.9886391432880521 for servers number: 1
Probability of loss: 0.9772765685967646 for servers number: 2
Probability of loss: 0.9658576339516565 for servers number: 3
Probability of loss: 0.95448196405488 for servers number: 4
Probability of loss: 0.9432578894942755 for servers number: 5
Probability of loss: 0.9317280012291499 for servers number: 6
Probability of loss: 0.9207105101924361 for servers number: 7
Probability of loss: 0.9092094472475922 for servers number: 8
Probability of loss: 0.8979776462263985 for servers number: 9
Probability of loss: 0.8866415769021416 for servers number: 10
Probability of loss: 0.8755066908765011 for servers number: 11
Probability of loss: 0.8643702307145228 for servers number: 12
Probability of loss: 0.8530634398571483 for servers number: 13
Probability of loss: 0.8426371025574335 for servers number: 14
Probability of loss: 0.8324559107642762 for servers number: 15
Probability of loss: 0.8223605344435883 for servers number: 16
Pro

In [12]:
requests_in_system_list = np.empty(0)

T = 10000
iterations = 1000
max_queue_size = 100
eps = 1e-1
betta = 0.95
x_betta = stats.norm.ppf(betta)

for i in trange(MIN_SERVERS, MAX_SERVERS + 1):    
    avarege_requests_in_system = num_in_system_iteration_resolver(
        T,
        iterations,
        i,
        max_queue_size,
        x_betta=x_betta,
        eps=eps,
        return_value = 'average_requests_in_system'
    )
    print(f'Avarege requests in system: {avarege_requests_in_system} for servers number: {i}')
    requests_in_system_list = np.append(requests_in_system_list, avarege_requests_in_system)

with open('./round-robin/requests_in_system_list.npy', 'wb') as f:
    np.save(f, requests_in_system_list)

  0%|          | 0/200 [00:00<?, ?it/s]

Avarege requests in system: 100.77041098197381 for servers number: 1
Avarege requests in system: 100.55700280239006 for servers number: 2
Avarege requests in system: 100.32909206447144 for servers number: 3
Avarege requests in system: 100.08187213101655 for servers number: 4
Avarege requests in system: 99.8080082699403 for servers number: 5
Avarege requests in system: 99.50157158976047 for servers number: 6
Avarege requests in system: 99.15834691648637 for servers number: 7
Avarege requests in system: 98.76777427033353 for servers number: 8
Avarege requests in system: 98.31239039493433 for servers number: 9
Avarege requests in system: 97.76475010570357 for servers number: 10
Avarege requests in system: 97.10069048708993 for servers number: 11
Avarege requests in system: 96.22646690685785 for servers number: 12
Avarege requests in system: 95.00039019533644 for servers number: 13
Avarege requests in system: 93.29963752839409 for servers number: 14
Avarege requests in system: 90.754851629